<a href="https://colab.research.google.com/github/mazen-Qaissi/Knowledge-Distillation/blob/main/task4_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install torchvision
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True, num_workers=2)

# train_loader = train_loader / 255.0
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=False, num_workers=2)
# test_loader = test_loader / 255.0

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self):
          super().__init__()
          self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3)
          self.pool = nn.MaxPool2d(2, 2)
          self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
          self.fc1 = nn.Linear(64*6*6,128)
          self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    # def __init__(self):
    #     super().__init__()
    #     self.network = nn.Sequential(
    #         nn.Conv2d(3, 32, kernel_size=3, padding=1),
    #         nn.ReLU(),
    #         nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

    #         nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    #         nn.ReLU(),
    #         nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
    #         nn.ReLU(),
    #         nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

    #         # nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    #         # nn.ReLU(),
    #         # nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
    #         # nn.ReLU(),
    #         # nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

    #         nn.Flatten(), 
    #         nn.Linear(128*8*8, 2048),
    #         nn.ReLU(),
    #         nn.Linear(2048, 512),
    #         nn.ReLU(),
    #         nn.Linear(512, 10))
        
    # def forward(self, xb):
    #     return self.network(xb)


net = Net()
count=sum(p.numel() for p in net.parameters() if p.requires_grad)

print(count)

315722


In [ ]:
accu = []
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            # print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(batch_size*(batch_idx+1))))
        accu.append(float(correct*100) / float(batch_size*(batch_idx+1)))

fit(net,train_loader)

Epoch : 0 [0/50000 (0%)]	Loss: 2.278438	 Accuracy:18.750%
Epoch : 0 [1600/50000 (3%)]	Loss: 2.193478	 Accuracy:21.446%
Epoch : 0 [3200/50000 (6%)]	Loss: 1.614233	 Accuracy:26.918%
Epoch : 0 [4800/50000 (10%)]	Loss: 1.763593	 Accuracy:30.050%
Epoch : 0 [6400/50000 (13%)]	Loss: 1.430788	 Accuracy:32.494%
Epoch : 0 [8000/50000 (16%)]	Loss: 1.648291	 Accuracy:34.300%
Epoch : 0 [9600/50000 (19%)]	Loss: 1.584668	 Accuracy:35.424%
Epoch : 0 [11200/50000 (22%)]	Loss: 1.469580	 Accuracy:36.841%
Epoch : 0 [12800/50000 (26%)]	Loss: 1.378423	 Accuracy:37.905%
Epoch : 0 [14400/50000 (29%)]	Loss: 1.293145	 Accuracy:38.747%
Epoch : 0 [16000/50000 (32%)]	Loss: 1.379140	 Accuracy:39.571%
Epoch : 0 [17600/50000 (35%)]	Loss: 1.321441	 Accuracy:40.438%
Epoch : 0 [19200/50000 (38%)]	Loss: 1.262761	 Accuracy:41.124%
Epoch : 0 [20800/50000 (42%)]	Loss: 1.159284	 Accuracy:41.955%
Epoch : 0 [22400/50000 (45%)]	Loss: 1.524726	 Accuracy:42.667%
Epoch : 0 [24000/50000 (48%)]	Loss: 1.166614	 Accuracy:43.276%
Epoch

In [ ]:
def evaluate(model):
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*32)))
    

# plt.plot(accu, label='accuracy')
# # plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

evaluate(net)


Test accuracy:0.704% 
